# Import Libraries

In [11]:
import torch, os
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import save_image
from torchvision.utils import make_grid
from torch.autograd import Variable
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch.utils import data as torchData

import sys
from os import listdir
from os.path import isfile, join
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image
import librosa
import librosa.display
import IPython.display as ipd

import random

%matplotlib inline

print(sys.version)
print('pytorch : ', torch.__version__)
print('librosa : ', librosa.__version__)
print('numpy : ', np.__version__)
print('scipy : ', sp.__version__)
print('matplotlib : ', mpl.__version__)

3.6.1 |Anaconda custom (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]
pytorch :  0.4.0
librosa :  0.5.1
numpy :  1.12.1
scipy :  1.0.1
matplotlib :  2.0.2


In [12]:
'''
Data Format : Time / Material / Action / Reaction

* Time : Hit time. ex) 2.677438

* Material
wood, metal, dirt, rock, leaf, plastic, cloth, paper, water, grass(top 10)
drywall, plastic bag, gravel, ceramic, glass, carpet, tile, other

* Action
hit, scratch

* Reaction
static, scatter, rigid-motion, deform, splash
'''

# Condsider top 10 materials / hit, scartch / static, scatter



'\nData Format : Time / Material / Action / Reaction\n\n* Time : Hit time. ex) 2.677438\n\n* Material\nwood, metal, dirt, rock, leaf, plastic, cloth, paper, water, grass(top 10)\ndrywall, plastic bag, gravel, ceramic, glass, carpet, tile, other\n\n* Action\nhit, scratch\n\n* Reaction\nstatic, scatter, rigid-motion, deform, splash\n'

# Data Split

In [16]:
path_dir = "./data/"
output_dir = "./data_split/"

file_list = os.listdir(path_dir) 

wav_files = [f for f in file_list if f[-12:] == "denoised.wav"]
txt_files = [f for f in file_list if f[-4:] == ".txt"]

wav_files.sort()
txt_files.sort()

print("wav files")
print(wav_files)

print("txt files")
print(txt_files)

data_list_1 = []
data_list_2 = []
data_list_3 = []

for path in txt_files:
    print(path)
    f = open(path_dir+path,'r')
    data =f.read()
    
    data_list = data.split("\n")
    data_list = data_list [:-1]
    data_list = [data for data in data_list if data.find("None",9) == -1]
    
    print(data_list)
    
    temp_hit = [float(data.split(" ")[0]) for data in data_list]
    data_list_1.append(temp_hit)
    data_list_2.append([data.split(" ")[1] for data in data_list])
    data_list_3.append([data.split(" ")[2] for data in data_list])
    
    f.close()
    
    
#for 1 wav file
for idx1 in range(len(data_list_1)):
    
    y, sr = librosa.load(path_dir+wav_files[idx1],sr=16000)
    
    for idx2 in range(len(data_list_1[idx1])):
        # 시간 계산
        curr_time = data_list_1[idx1][idx2]
        output_path = output_dir + txt_files[idx1][:-9]+data_list_2[idx1][idx2]+"_"+data_list_3[idx1][idx2]+"_"+str(idx1)+"_"+str(idx2)+".wav"
#        print(output_path)
        #Normalize [-1,1]
        temp = y[int(sr*curr_time):int(sr*curr_time)+16384]
        max_amp = np.max(np.abs(temp))
        if max_amp > 1:
            temp /= max_amp
        librosa.output.write_wav(output_path, temp,sr)

wav files


['2015-02-16-16-49-06_denoised.wav', '2015-02-16-16-56-35_denoised.wav', '2015-02-16-16-58-57_denoised.wav', '2015-02-16-17-02-05_denoised.wav', '2015-02-16-17-03-55_denoised.wav', '2015-02-16-17-08-51_denoised.wav', '2015-02-16-17-15-23_denoised.wav', '2015-02-16-17-27-53_denoised.wav', '2015-02-16-17-32-18_denoised.wav', '2015-02-16-17-35-07_denoised.wav', '2015-02-16-17-40-35_denoised.wav', '2015-02-16-17-46-48_denoised.wav', '2015-02-16-17-52-15_denoised.wav', '2015-02-16-17-54-47_denoised.wav', '2015-02-16-18-02-38_denoised.wav', '2015-02-16-18-07-52_denoised.wav', '2015-02-21-17-08-25_denoised.wav', '2015-02-21-17-09-17_denoised.wav', '2015-02-21-17-13-35_denoised.wav', '2015-02-21-17-14-45_denoised.wav', '2015-02-21-17-16-04_denoised.wav', '2015-02-21-17-18-56_denoised.wav', '2015-02-21-17-27-48_denoised.wav', '2015-02-21-17-29-20_denoised.wav', '2015-02-21-17-31-46_denoised.wav', '2015-02-21-17-33-03_denoised.wav', '2015-02-21-17-34-22_denoised.wav', '2015-02-21-17-37-20_denois

txt files


['2015-02-16-16-49-06_times.txt', '2015-02-16-16-56-35_times.txt', '2015-02-16-16-58-57_times.txt', '2015-02-16-17-02-05_times.txt', '2015-02-16-17-03-55_times.txt', '2015-02-16-17-08-51_times.txt', '2015-02-16-17-15-23_times.txt', '2015-02-16-17-27-53_times.txt', '2015-02-16-17-32-18_times.txt', '2015-02-16-17-35-07_times.txt', '2015-02-16-17-40-35_times.txt', '2015-02-16-17-46-48_times.txt', '2015-02-16-17-52-15_times.txt', '2015-02-16-17-54-47_times.txt', '2015-02-16-18-02-38_times.txt', '2015-02-16-18-07-52_times.txt', '2015-02-21-17-08-25_times.txt', '2015-02-21-17-09-17_times.txt', '2015-02-21-17-13-35_times.txt', '2015-02-21-17-14-45_times.txt', '2015-02-21-17-16-04_times.txt', '2015-02-21-17-18-56_times.txt', '2015-02-21-17-27-48_times.txt', '2015-02-21-17-29-20_times.txt', '2015-02-21-17-31-46_times.txt', '2015-02-21-17-33-03_times.txt', '2015-02-21-17-34-22_times.txt', '2015-02-21-17-37-20_times.txt', '2015-02-21-17-39-27_times.txt', '2015-02-21-17-41-06_times.txt', '2015-02-

2015-02-16-16-49-06_times.txt
['2.677438 ceramic hit static', '3.675198 ceramic hit static', '4.713354 ceramic hit static', '6.554396 ceramic hit static', '7.496979 ceramic hit rigid-motion', '8.427156 ceramic hit static', '9.382239 ceramic hit static', '10.410521 metal hit rigid-motion', '12.281136 ceramic hit rigid-motion', '13.104990 water hit splash', '14.151709 ceramic hit rigid-motion', '20.941792 ceramic hit static', '21.905563 ceramic hit static', '23.108355 ceramic scratch static', '24.642521 ceramic scratch static', '28.452520 metal hit static', '29.441355 metal hit static', '31.693365 metal scratch static', '41.459679 metal hit static', '44.055698 ceramic hit static', '44.897221 ceramic hit rigid-motion', '45.738251 ceramic scratch static', '47.434917 ceramic hit static', '48.176094 metal hit rigid-motion', '49.101509 ceramic hit static', '50.091167 ceramic hit static', '50.945114 ceramic hit static', '51.807407 ceramic hit rigid-motion', '52.705387 ceramic scratch static', 

...

['1.448917 gravel scratch scatter', '2.016417 gravel scratch scatter', '2.611833 gravel scratch scatter', '3.839271 gravel hit deform', '4.424802 gravel hit scatter', '5.555979 gravel hit scatter', '6.048604 gravel scratch scatter', '7.134240 gravel hit scatter', '7.433000 gravel scratch scatter', '7.777083 gravel scratch scatter', '8.862010 gravel scratch scatter', '11.832313 gravel scratch scatter', '12.402354 gravel scratch scatter', '12.864635 gravel scratch scatter', '13.278469 gravel scratch scatter', '14.009542 gravel scratch scatter', '15.791177 gravel hit scatter', '16.809042 dirt hit scatter', '17.066000 dirt hit scatter', '17.329042 gravel scratch scatter', '17.832041 gravel scratch scatter']
2015-10-06-19-38-24-32_times.txt
['1.464594 leaf scratch scatter', '2.544510 dirt scratch scatter', '3.135115 leaf scratch scatter', '3.780406 leaf hit rigid-motion', '4.329979 leaf hit deform', '4.904625 leaf hit rigid-motion', '5.375406 leaf scratch rigid-motion', '5.926875 leaf scrat

['2.237021 water scratch splash', '2.636354 water scratch splash', '3.051510 water scratch splash', '3.617875 water scratch splash', '4.319750 water scratch splash', '4.643198 water scratch splash', '5.532063 water scratch splash', '6.468146 water scratch splash', '7.854729 water scratch splash', '11.694542 water scratch splash', '15.272688 water scratch splash', '16.044209 water scratch splash', '16.627094 water scratch splash', '16.890219 water scratch splash', '17.691458 water scratch splash', '18.244740 water scratch splash', '19.684969 water scratch splash', '20.662678 water scratch splash', '21.621708 water hit splash', '22.609896 water scratch splash', '23.168980 water scratch splash', '23.538729 water hit splash', '26.564135 water scratch splash', '27.200874 water scratch splash', '28.206688 water scratch splash', '29.085197 water scratch splash', '29.443052 water scratch splash', '29.705626 water scratch splash', '30.303938 water hit splash', '30.610760 water scratch splash', 

['1.414229 dirt scratch scatter', '2.026562 dirt scratch scatter', '3.875979 dirt hit scatter', '4.982917 dirt scratch scatter', '6.769354 dirt scratch scatter', '7.327635 dirt scratch scatter', '7.870031 dirt hit scatter', '8.423896 dirt hit scatter', '8.924645 dirt scratch scatter', '9.936344 dirt scratch scatter', '10.963583 dirt scratch scatter', '11.505750 dirt scratch scatter', '12.590604 dirt hit scatter', '13.622719 dirt scratch scatter', '14.161125 dirt scratch scatter', '14.651719 dirt hit scatter', '16.322239 dirt scratch scatter', '16.868385 dirt scratch scatter', '17.388124 dirt scratch scatter', '17.916021 dirt scratch scatter', '18.487032 dirt scratch scatter', '19.026594 dirt scratch scatter', '20.076521 dirt scratch scatter', '22.654875 dirt scratch scatter', '23.384699 dirt scratch scatter', '23.996010 dirt hit scatter', '25.187708 dirt scratch scatter', '25.748156 dirt scratch scatter', '26.312864 dirt scratch scatter', '27.424114 dirt scratch scatter', '28.021864 di

['1.498562 water scratch splash', '2.149333 water hit deform', '2.809062 water hit splash', '3.076864 water scratch splash', '3.419708 water scratch splash', '3.767958 water hit splash', '4.103385 water hit splash', '4.394094 water scratch splash', '4.756708 water scratch splash', '6.127906 water scratch splash', '6.744187 water scratch splash', '7.392323 water scratch splash', '7.674708 water hit splash', '8.053073 water scratch splash', '9.394000 rock hit splash', '11.423020 water scratch splash', '12.089948 water hit splash', '14.900469 rock hit static', '15.530896 water scratch splash', '16.076876 water hit splash', '16.895073 water scratch splash', '17.188322 water scratch splash', '17.518917 water scratch splash', '17.898937 water hit splash', '18.589949 water hit splash', '20.200970 water scratch splash', '20.569199 water scratch splash', '20.859541 water hit splash', '21.241646 water hit splash', '21.497999 water scratch splash', '21.815886 water scratch splash', '24.604687 wat

['1.070948 water hit splash', '1.627135 water hit splash', '1.980406 water hit splash', '2.871354 water hit splash', '3.142115 water hit splash', '4.164208 water hit splash', '5.047333 water hit splash', '5.437990 water hit splash', '7.012312 water hit splash', '7.410208 water hit splash', '7.664458 water hit splash', '8.059896 water hit splash', '8.350041 water hit splash', '8.668667 water scratch splash', '8.953448 water hit splash', '9.256761 water hit splash', '9.884427 water hit splash', '10.153427 water hit splash', '11.265407 water hit splash', '12.168917 water hit splash', '12.937115 water hit splash', '13.244657 water hit splash', '13.873552 water hit splash', '14.132614 water hit splash', '14.435010 water hit splash', '15.072906 water hit splash', '15.364177 water scratch splash', '16.829178 rock hit splash', '17.682823 water hit splash', '18.002197 water hit splash', '18.567865 water hit splash', '19.147655 water hit splash', '19.422739 water hit splash', '19.737574 water hi

# Data Load

In [17]:
path_dir = "./data_split/"

In [18]:

class AudioLoader(torchData.Dataset):
    
    def __init__(self, inPath, isShuffle = False):
        
        files = os.listdir(inPath)
        
        files = [f for f in files if f[-4:] == ".wav"]
        
#        files = [f for f in files if (f.find("wood",18) != -1 or f.find("metal",18) != -1)]
        files = [f for f in files if (f.find("ceramic",18) != -1 or f.find("metal",18) != -1)]
        
        if isShuffle:
            random.shuffle(files)
        
        self.inPath = inPath
        self.isShuffle = isShuffle
        self.len = len(files)  
        self.files = files
        print("# of wav files : ", self.len)
    
    def __getitem__(self, idx):
        
        y, sr = librosa.load(self.inPath+self.files[idx], 16000)
        y = torch.from_numpy(y)

        return y
    
    def __len__(self):
        return self.len
    
    
    
    

In [19]:
!python main.py --gan_type="WaveGAN" --benchmark_mode=True --gpu_mode=True --epoch=10 --batch_size=5 --input_size=16384 --data_dir=data_split --isShuffle=False

Namespace(batch_size=5, benchmark_mode=True, beta1=0.5, beta2=0.999, data_dir='data_split', dataset='mnist', epoch=10, gan_type='WaveGAN', gpu_mode=True, input_size=16384, isShuffle='False', log_dir='logs', lrD=0.0002, lrG=0.0002, result_dir='results', save_dir='models', split='')
# of wav files :  4402
---------- Networks architecture -------------
WaveGAN_Generator(
  (fc): Linear(in_features=100, out_features=16384, bias=True)
  (deconv1): Sequential(
    (0): ConvTranspose1d(1024, 512, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (deconv2): Sequential(
    (0): ConvTranspose1d(512, 256, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (deconv3): Sequential(
    (0): ConvTranspose1d(256, 128, kernel_size=(25,), stride=(4,), padding=(11,), output_padding

THCudaCheck FAIL file=c:\programdata\miniconda3\conda-bld\pytorch_1524546371102\work\aten\src\thc\generic/THCStorage.cu line=58 error=2 : out of memory
Traceback (most recent call last):
  File "main.py", line 84, in <module>
    main()
  File "main.py", line 76, in main
    gan.train()
  File "C:\Users\Chang Minwook\Documents\pytorch-WaveGAN\wavegan.py", line 270, in train
    D_loss.backward()
  File "C:\Users\Chang Minwook\Anaconda3\lib\site-packages\torch\tensor.py", line 93, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "C:\Users\Chang Minwook\Anaconda3\lib\site-packages\torch\autograd\__init__.py", line 89, in backward
    allow_unreachable=True)  # allow_unreachable flag
RuntimeError: cuda runtime error (2) : out of memory at c:\programdata\miniconda3\conda-bld\pytorch_1524546371102\work\aten\src\thc\generic/THCStorage.cu:58


In [20]:
!python main.py --gan_type="WaveGAN" --benchmark_mode=True --gpu_mode=True --epoch=20 --batch_size=5 --input_size=16384 --data_dir=data_split --isShuffle=False

Namespace(batch_size=5, benchmark_mode=True, beta1=0.5, beta2=0.999, data_dir='data_split', dataset='mnist', epoch=20, gan_type='WaveGAN', gpu_mode=True, input_size=16384, isShuffle='False', log_dir='logs', lrD=0.0002, lrG=0.0002, result_dir='results', save_dir='models', split='')
# of wav files :  4402
---------- Networks architecture -------------
WaveGAN_Generator(
  (fc): Linear(in_features=100, out_features=16384, bias=True)
  (deconv1): Sequential(
    (0): ConvTranspose1d(1024, 512, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (deconv2): Sequential(
    (0): ConvTranspose1d(512, 256, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (deconv3): Sequential(
    (0): ConvTranspose1d(256, 128, kernel_size=(25,), stride=(4,), padding=(11,), output_padding

THCudaCheck FAIL file=c:\programdata\miniconda3\conda-bld\pytorch_1524546371102\work\aten\src\thc\generic/THCStorage.cu line=58 error=2 : out of memory
Traceback (most recent call last):
  File "main.py", line 84, in <module>
    main()
  File "main.py", line 76, in main
    gan.train()
  File "C:\Users\Chang Minwook\Documents\pytorch-WaveGAN\wavegan.py", line 270, in train
    D_loss.backward()
  File "C:\Users\Chang Minwook\Anaconda3\lib\site-packages\torch\tensor.py", line 93, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "C:\Users\Chang Minwook\Anaconda3\lib\site-packages\torch\autograd\__init__.py", line 89, in backward
    allow_unreachable=True)  # allow_unreachable flag
RuntimeError: cuda runtime error (2) : out of memory at c:\programdata\miniconda3\conda-bld\pytorch_1524546371102\work\aten\src\thc\generic/THCStorage.cu:58


In [21]:
!python main.py --gan_type="WaveGAN" --benchmark_mode=True --gpu_mode=True --epoch=30 --batch_size=3 --input_size=16384 --data_dir=data_split --isShuffle=False

Namespace(batch_size=3, benchmark_mode=True, beta1=0.5, beta2=0.999, data_dir='data_split', dataset='mnist', epoch=30, gan_type='WaveGAN', gpu_mode=True, input_size=16384, isShuffle='False', log_dir='logs', lrD=0.0002, lrG=0.0002, result_dir='results', save_dir='models', split='')
# of wav files :  4402
---------- Networks architecture -------------
WaveGAN_Generator(
  (fc): Linear(in_features=100, out_features=16384, bias=True)
  (deconv1): Sequential(
    (0): ConvTranspose1d(1024, 512, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (deconv2): Sequential(
    (0): ConvTranspose1d(512, 256, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (deconv3): Sequential(
    (0): ConvTranspose1d(256, 128, kernel_size=(25,), stride=(4,), padding=(11,), output_padding

THCudaCheck FAIL file=c:\programdata\miniconda3\conda-bld\pytorch_1524546371102\work\aten\src\thc\generic/THCStorage.cu line=58 error=2 : out of memory
Traceback (most recent call last):
  File "main.py", line 84, in <module>
    main()
  File "main.py", line 76, in main
    gan.train()
  File "C:\Users\Chang Minwook\Documents\pytorch-WaveGAN\wavegan.py", line 270, in train
    D_loss.backward()
  File "C:\Users\Chang Minwook\Anaconda3\lib\site-packages\torch\tensor.py", line 93, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "C:\Users\Chang Minwook\Anaconda3\lib\site-packages\torch\autograd\__init__.py", line 89, in backward
    allow_unreachable=True)  # allow_unreachable flag
RuntimeError: cuda runtime error (2) : out of memory at c:\programdata\miniconda3\conda-bld\pytorch_1524546371102\work\aten\src\thc\generic/THCStorage.cu:58


# Hparams Setting

In [5]:
lr = 0.0001
max_epoch = 20
batch_size = 64
z_dim = 100
image_size = 64
g_conv_dim = 64
d_conv_dim = 64
log_step = 100
sample_step = 500
sample_num = 32

# Generator

In [6]:
# Conv2d (Batch_num , Channel, length) 
def deconv(c_in, c_out, k_size, stride=2, pad=1, bn=True):
    """Custom deconvolutional layer for simplicity."""
    layers = []
    layers.append(nn.ConvTranspose2d(c_in, c_out, k_size, stride, pad))
    if bn:
        layers.append(nn.BatchNorm2d(c_out))
    return nn.Sequential(*layers)

#deconv1d
def deconv1d(c_in, c_out, k_size, stride=4, pad=11, out_pad = 1,bn=True):
    """Custom convolutional 1d lyaer for simplicity."""
    layers = []
    layers.append(nn.ConvTranspose1d(c_in, c_out, k_size, stride, pad,out_pad))
    if bn:
        layers.append(nn.BatchNorm1d(c_out))
    return nn.Sequential(*layers)

class WaveGAN_Generator(nn.Module):
    """Generator containing 7 deconvolutional layers."""
    def __init__(self, z_dim=100, image_size=128, conv_dim=g_conv_dim):
        super(WaveGAN_Generator, self).__init__()
        self.fc = nn.Linear(z_dim, 256*conv_dim)
#        self.deconv1 = deconv1d(conv_dim*16, conv_dim*8, (25,2,2), pad = 11)
        self.deconv1 = deconv1d(conv_dim*16, conv_dim*8, k_size = 25, pad = 11, out_pad = 1)
        self.deconv2 = deconv1d(conv_dim*8, conv_dim*4, 25)
        self.deconv3 = deconv1d(conv_dim*4, conv_dim*2, 25)
        self.deconv4 = deconv1d(conv_dim*2, conv_dim, 25)
        self.deconv5 = deconv1d(conv_dim, 1, 25, bn=False)
        
    def forward(self, z):
#        z = z.view(z.size(0), z.size(1))      # If image_size is 64, output shape is as below.
        out = self.fc(z)                 # (?, 256d)
        print(out.size())
        out = out.view(out.size(0),16*g_conv_dim,16 ) # (?,16,16d)
        print(out.size())
        out = F.relu(out)
        out = F.relu(self.deconv1(out))  # (?, 64, 8d)
        print("a")
        print(out.size())
        
        out = F.relu(self.deconv2(out))  # (?, 256, 4d)
        
        print(out.size())
        out = F.relu(self.deconv3(out))  # (?, 1024, 2d)
        print(out.size())
        out = F.relu(self.deconv4(out))  # (?, 4096, d)
        print(out.size())
        out = F.tanh(self.deconv5(out))  # (?, 16384, c)
        print(out.size())
        return out
    
G = WaveGAN_Generator(z_dim,image_size,g_conv_dim)
G

WaveGAN_Generator(
  (fc): Linear(in_features=100, out_features=16384, bias=True)
  (deconv1): Sequential(
    (0): ConvTranspose1d(1024, 512, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (deconv2): Sequential(
    (0): ConvTranspose1d(512, 256, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (deconv3): Sequential(
    (0): ConvTranspose1d(256, 128, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (deconv4): Sequential(
    (0): ConvTranspose1d(128, 64, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (deconv5): Sequential(
    (0

In [7]:
n_data = torch.ones(5,100)
#n_data

In [8]:
G = WaveGAN_Generator()

output_ = G.forward(n_data)
output_.size()[0]
output_.view(output_.size()[0],1,output_.size()[2])

torch.Size([5, 16384])
torch.Size([5, 1024, 16])
a
torch.Size([5, 512, 64])
torch.Size([5, 256, 256])
torch.Size([5, 128, 1024])
torch.Size([5, 64, 4096])
torch.Size([5, 1, 16384])


tensor([[[-3.2387e-02, -4.7821e-02,  3.9117e-04,  ...,  3.1392e-02,
          -3.6386e-02,  3.2837e-02]],

        [[-3.2387e-02, -4.7821e-02,  3.9117e-04,  ...,  3.1392e-02,
          -3.6386e-02,  3.2837e-02]],

        [[-3.2387e-02, -4.7821e-02,  3.9117e-04,  ...,  3.1392e-02,
          -3.6386e-02,  3.2837e-02]],

        [[-3.2387e-02, -4.7821e-02,  3.9117e-04,  ...,  3.1392e-02,
          -3.6386e-02,  3.2837e-02]],

        [[-3.2387e-02, -4.7821e-02,  3.9117e-04,  ...,  3.1392e-02,
          -3.6386e-02,  3.2837e-02]]])

# Discriminator

In [9]:
def conv(c_in, c_out, k_size, stride=4, pad=1,bn=True):
    """Custom convolutional 1d lyaer for simplicity."""
    layers = []
    layers.append(nn.Conv2d(c_in, c_out, k_size,stride,pad))
    if bn:
        layers.append(nn.BatchNorm2d(c_out))
    return nn.Sequential(*layers)


def conv1d(c_in, c_out, k_size, stride=4, pad=11, out_pad = 1,bn=True):
    """Custom convolutional 1d lyaer for simplicity."""
    layers = []
    layers.append(nn.Conv1d(c_in, c_out, k_size,stride,pad))
    if bn:
        layers.append(nn.BatchNorm1d(c_out))
    return nn.Sequential(*layers)

def apply_phaseshuffle(x, n_phase, pad_type = 'refelct'):
    (batch, n_channel, x_len) = x.shape
    r = random.randrange(-n_phase, n_phase+1)
    pad_l = np.maximum( r , 0)
    pad_r = np.maximum(-r, 0)
    phase_start = pad_r
    
    padding = nn.ReflectionPad2d((pad_l, pad_r, 0, 0))
#    print("phase : ", r)
    #print("pad_l, pad_r, phase_start, x_len", pad_l, pad_r, phase_start, x_len)
    #print("x.shape", x.shape)
    
    for x_ in x:

        ch_, len_ = x_.shape
        x_ = x_.reshape(1,1,ch_,len_)
        x_ = padding(x_)
        x_ = x_[:, :, :,phase_start:phase_start + len_]
        x_ = x_.reshape(ch_,len_)
    
    return x
    

#Ref DCGAN : https://github.com/InsuJeon/Hello-Generative-Model/blob/master/Day04/DCGAN/dcgan.ipynb
class WaveGAN_Discriminator(nn.Module):
    """Discriminator containing 4 convolutional layers."""
    

    def __init__(self,  conv_dim=d_conv_dim, n_phase = 2):
        
        if n_phase > 0:
            self.phaseshuffle = lambda x: apply_phaseshuffle(x,n_phase)
        else:
            self.phaseshuffle = lambda x: x
        
        super(WaveGAN_Discriminator, self).__init__()
        self.conv1 = conv1d(1, conv_dim, 25, bn=False)
        self.conv2 = conv1d(conv_dim, conv_dim*2, 25)
        self.conv3 = conv1d(conv_dim*2, conv_dim*4, 25)
        self.conv4 = conv1d(conv_dim*4, conv_dim*8, 25)
        self.conv5 = conv1d(conv_dim*8, conv_dim*16, 25)
        self.fc = nn.Linear(conv_dim*16*16,1)
        
#            conv(conv_dim*8, 1, int(image_size/16), 1, 0, False)
        
    def forward(self, x):
        # (?, 1, 16384) -> (?, 64, 4096)
        out = F.leaky_relu(self.conv1(x), 0.2)    # (?, 64, 32, 32)
        out = self.phaseshuffle(out)
        
        # (?, 64, 4096) -> (?, 128, 1024)
        out = F.leaky_relu(self.conv2(out), 0.2)  # (?, 128, 16, 16)
        out = self.phaseshuffle(out)

        # (?, 128, 1024) -> (?, 256, 256)
        out = F.leaky_relu(self.conv3(out), 0.2)  # (?, 256, 8, 8)
        out = self.phaseshuffle(out)

        # (?, 256, 256) -> (?, 512, 64)
        out = F.leaky_relu(self.conv4(out), 0.2)  # (?, 512, 4, 4)
        out = self.phaseshuffle(out)

        # (?, 512, 64) -> (?, 1024, 16)
        out = F.leaky_relu(self.conv5(out), 0.2)  # (?, 512, 4, 4)
        out = self.phaseshuffle(out)

        # (?, 1024, 16) -> (?, 16384)
        # (?,16384) -> (?,1)
        out = out.view(out.size(0), 256 * d_conv_dim)
        out = F.sigmoid(self.fc(out))
        out = out.squeeze()
        
        return out

In [10]:
D = WaveGAN_Discriminator(n_phase = 2)
D.cuda()

WaveGAN_Discriminator(
  (conv1): Sequential(
    (0): Conv1d(1, 64, kernel_size=(25,), stride=(4,), padding=(11,))
  )
  (conv2): Sequential(
    (0): Conv1d(64, 128, kernel_size=(25,), stride=(4,), padding=(11,))
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): Sequential(
    (0): Conv1d(128, 256, kernel_size=(25,), stride=(4,), padding=(11,))
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv4): Sequential(
    (0): Conv1d(256, 512, kernel_size=(25,), stride=(4,), padding=(11,))
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv5): Sequential(
    (0): Conv1d(512, 1024, kernel_size=(25,), stride=(4,), padding=(11,))
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc): Linear(in_features=16384, out_features=1, bias=True)
)

In [16]:
print(output.size)
D.forward(output)

<built-in method size of Tensor object at 0x000001FCF4280C60>


tensor([ 0.4828,  0.4828,  0.4828,  0.4828,  0.4828])

In [103]:
print("path_dir : ", path_dir)
dataset = AudioLoader(path_dir)

trainLoader = torchData.DataLoader(
    dataset = dataset,
    batch_size = 3,
    shuffle = False
)

criterion_vanillia = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=lr, betas=(0.5,0.9))
g_optimizer = torch.optim.Adam(G.parameters(), lr=lr, betas=(0.5,0.9))


for epoch in range(10):
    
    for idx, data in enumerate(trainLoader):
        #print(data)

path_dir :  ./data_split/
# of wav files :  38
tensor([[-0.0582,  0.0627,  0.0343,  ..., -0.0002, -0.0002, -0.0002],
        [ 0.0146,  0.0841, -0.2565,  ...,  0.0249,  0.0085, -0.0068],
        [-0.1053,  0.3532, -0.4445,  ..., -0.0004, -0.0003, -0.0002]])
tensor([[-0.2800,  0.4602, -0.0682,  ..., -0.7203, -0.2984,  0.1931],
        [ 0.0158, -0.2845,  0.6770,  ...,  0.0211,  0.0029,  0.0449],
        [-0.2772, -0.1096,  0.1559,  ...,  0.0000, -0.0000,  0.0000]])
tensor([[ 0.0702, -0.5358,  0.3916,  ..., -0.0011, -0.0004,  0.0015],
        [ 0.0638, -0.0955, -0.0287,  ...,  0.0033,  0.0052, -0.0081],
        [-0.1652, -0.0455,  0.5964,  ..., -0.0002, -0.0003, -0.0001]])
tensor([[ 0.0049, -0.0666,  0.0408,  ..., -0.0014,  0.0000,  0.0007],
        [-0.1692,  0.1627,  0.1360,  ...,  0.1122,  0.0911, -0.0940],
        [-0.0904, -0.0285,  0.2533,  ...,  0.0005,  0.0026, -0.0013]])
tensor([[ 0.0717, -0.5103,  0.3428,  ...,  0.0003, -0.0015, -0.0028],
        [-0.1422,  0.0434,  0.7737,  ..

        [-0.1053,  0.3532, -0.4445,  ..., -0.0004, -0.0003, -0.0002]])
tensor([[-0.2800,  0.4602, -0.0682,  ..., -0.7203, -0.2984,  0.1931],
        [ 0.0158, -0.2845,  0.6770,  ...,  0.0211,  0.0029,  0.0449],
        [-0.2772, -0.1096,  0.1559,  ...,  0.0000, -0.0000,  0.0000]])
tensor([[ 0.0702, -0.5358,  0.3916,  ..., -0.0011, -0.0004,  0.0015],
        [ 0.0638, -0.0955, -0.0287,  ...,  0.0033,  0.0052, -0.0081],
        [-0.1652, -0.0455,  0.5964,  ..., -0.0002, -0.0003, -0.0001]])
tensor([[ 0.0049, -0.0666,  0.0408,  ..., -0.0014,  0.0000,  0.0007],
        [-0.1692,  0.1627,  0.1360,  ...,  0.1122,  0.0911, -0.0940],
        [-0.0904, -0.0285,  0.2533,  ...,  0.0005,  0.0026, -0.0013]])
tensor([[ 0.0717, -0.5103,  0.3428,  ...,  0.0003, -0.0015, -0.0028],
        [-0.1422,  0.0434,  0.7737,  ...,  0.0006,  0.0002,  0.0007],
        [-0.1921,  0.3932, -0.3010,  ...,  0.0074,  0.0038, -0.0114]])
tensor([[-0.3336,  0.1196, -0.2098,  ...,  0.0000,  0.0000,  0.0000],
        [-0.124

        [-0.2772, -0.1096,  0.1559,  ...,  0.0000, -0.0000,  0.0000]])
tensor([[ 0.0702, -0.5358,  0.3916,  ..., -0.0011, -0.0004,  0.0015],
        [ 0.0638, -0.0955, -0.0287,  ...,  0.0033,  0.0052, -0.0081],
        [-0.1652, -0.0455,  0.5964,  ..., -0.0002, -0.0003, -0.0001]])
tensor([[ 0.0049, -0.0666,  0.0408,  ..., -0.0014,  0.0000,  0.0007],
        [-0.1692,  0.1627,  0.1360,  ...,  0.1122,  0.0911, -0.0940],
        [-0.0904, -0.0285,  0.2533,  ...,  0.0005,  0.0026, -0.0013]])
tensor([[ 0.0717, -0.5103,  0.3428,  ...,  0.0003, -0.0015, -0.0028],
        [-0.1422,  0.0434,  0.7737,  ...,  0.0006,  0.0002,  0.0007],
        [-0.1921,  0.3932, -0.3010,  ...,  0.0074,  0.0038, -0.0114]])
tensor([[-0.3336,  0.1196, -0.2098,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1248,  0.1481, -0.0112,  ..., -0.0000, -0.0002, -0.0001],
        [-0.0023, -0.0301,  0.0789,  ..., -0.0045,  0.0023,  0.0035]])
tensor([[-0.0582,  0.0627,  0.0343,  ..., -0.0002, -0.0002, -0.0002],
        [-0.216

        [-0.1652, -0.0455,  0.5964,  ..., -0.0002, -0.0003, -0.0001]])
tensor([[ 0.0049, -0.0666,  0.0408,  ..., -0.0014,  0.0000,  0.0007],
        [-0.1692,  0.1627,  0.1360,  ...,  0.1122,  0.0911, -0.0940],
        [-0.0904, -0.0285,  0.2533,  ...,  0.0005,  0.0026, -0.0013]])
tensor([[ 0.0717, -0.5103,  0.3428,  ...,  0.0003, -0.0015, -0.0028],
        [-0.1422,  0.0434,  0.7737,  ...,  0.0006,  0.0002,  0.0007],
        [-0.1921,  0.3932, -0.3010,  ...,  0.0074,  0.0038, -0.0114]])
tensor([[-0.3336,  0.1196, -0.2098,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1248,  0.1481, -0.0112,  ..., -0.0000, -0.0002, -0.0001],
        [-0.0023, -0.0301,  0.0789,  ..., -0.0045,  0.0023,  0.0035]])
tensor([[-0.0582,  0.0627,  0.0343,  ..., -0.0002, -0.0002, -0.0002],
        [-0.2166,  0.2974, -0.1138,  ...,  0.0098, -0.0104, -0.0006],
        [-0.2386,  0.1556, -0.1297,  ...,  0.0182,  0.1056, -0.0516]])
tensor([[ 0.0459,  0.0499, -0.1558,  ..., -0.0026, -0.0008,  0.0000],
        [-0.340